# Exploratory Testing Notebook for Asset Time Machine App

In [1]:
# base packages
import numpy as np

## Testing Data Connections

### Pulling from Yahoo Finance's API

In [2]:
import fix_yahoo_finance as yf

# pulling historical data for Apple using fixed yahoo finance API
initial_data_pull_df = yf.download(
    tickers='AAPL',
    start='2016-01-01',
    end='2018-01-01',
    progress=False
)

initial_data_pull_df.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-12-22,174.68,175.42,174.50,175.01,171.03,16114600
2017-12-26,170.80,171.47,169.68,170.57,166.69,33185500
2017-12-27,170.10,170.78,169.71,170.60,166.72,21498200
2017-12-28,171.00,171.85,170.48,171.08,167.19,16480200
2017-12-29,170.52,170.59,169.22,169.23,165.38,25884400


## Data Processing

### Trimming to Only Needed Data

In [3]:
# making date a column instead of an index
initial_data_pull_df.reset_index(level=0, inplace=True)

# keeping just date and adjusted closing value
cleaned_close_df = initial_data_pull_df.loc[:,('Date','Adj Close')]

In [4]:
cleaned_close_df.tail()

,Date,Adj Close
498,2017-12-22,171.03
499,2017-12-26,166.69
500,2017-12-27,166.72
501,2017-12-28,167.19
502,2017-12-29,165.38


### Finding % Down from ATH for Each Value

In [18]:
# need to find an efficient and pythonic way to, for each date, find the % down from all-time high

# instantiate empty lists in which to store ATH details
recent_ath_date = []
recent_ath_val  = []

for date in cleaned_close_df.loc[:,'Date']:
    
    # create df of time period to evaluate (all closes prior to chosen date)
    period_to_eval_df = cleaned_close_df.loc[cleaned_close_df.loc[:,'Date'] < date, :]

    # check the period to evaluate for ATH and get the date and value
    recent_ath_row = period_to_eval_df.loc[period_to_eval_df.loc[:,'Adj Close'] == period_to_eval_df.loc[:,'Adj Close'].max(), :]

    recent_ath_date.append(recent_ath_row.loc[:,'Date'])
    recent_ath_val.append(recent_ath_row.loc[:,'Adj Close'])  

In [19]:
recent_ath_row

,Date,Adj Close
494,2017-12-18,172.4


In [34]:
recent_ath_date.pop(1).item()

1451865600000000000

In [10]:
recent_ath_row.loc[:,'Adj Close'].values[0]

172.4

In [ ]:
# adding ATH info to main df
cleaned_close_df['recent_ath_date'] = recent_ath_date

In [ ]:
cleaned_close_df.head()